# 1. Setup AI Fact Sheets

get the api key from your ibm-cloud interface. 

api_key = 'xxx'

In [ ]:
# The code was removed by Watson Studio for sharing.

insert the project token by clicking "insert project token" behind the three dots in the top right corner. You might have to create process token first in the cp4d/project/manage/access_control interface

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
import os

try:
    from ibm_aigov_facts_client import AIGovFactsClient
except:
    !pip install -U ibm-aigov-facts-client
    from ibm_aigov_facts_client import AIGovFactsClient
        
        
PROJECT_UID= os.environ['PROJECT_ID'] #this assumes you are running in Studio. When running externally please add the project id here.
CPD_URL=os.environ['RUNTIME_ENV_APSX_URL'][len('https://api.'):] #the variable starts with https://api." and you only need the rest 'dataplatform.dev.cloud.ibm.com'
CONTAINER_ID=PROJECT_UID
CONTAINER_TYPE='project'
EXPERIMENT_NAME='car_insurance_outcome_prediction'
# CPD4D API key. Navigate to hamburger menu -> Acess (IAM) -> API Keys to create one for you (or have your account admin do it)

# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
PROJECT_ACCESS_TOKEN=project.project_context.accessToken.replace('Bearer ','') #You can create or lookup such a token from the settings tab of this project



facts_client = AIGovFactsClient(api_key=api_key,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

# 2. Train Model

### using Pipeline 4 Notebook - AutoAI Notebook v1.17.1
![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)

<a id="install"></a>
### Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn


In [ ]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install autoai-libs==1.14.9 | tail -n 1
!pip install scikit-learn==1.1.1 | tail -n 1

<a id="variables_definition"></a>
### AutoAI experiment metadata 
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

copy this part from the auto AI notebook
```
training_data_references = [
    DataConnection(
        data_asset_id='de66f9ac-2117-4f39-bf15-ba4cee99a4bc'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='xxx',
        model_location='xxx',
        training_status='xxx'
    )
)
```

In [43]:
# The code was removed by Watson Studio for sharing.

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='OUTCOME',
    holdout_size=0.1,
    scoring='accuracy',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='c7354a00-b66b-4896-8443-40b1ac51c846',
    positive_label='0.0',
    drop_duplicates=True
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)

### Read and preprocess data

In [ ]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=False)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TNoOp
from sklearn.ensemble import RandomForestClassifier

#### Pre-processing & Estimator.

In [ ]:
column_selector_0 = ColumnSelector(
    columns_indices_list=[
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str", "char_str", "char_str", "char_str", "char_str",
        "char_str", "float_int_num", "char_str", "float_int_num",
        "float_int_num", "int_num", "float_int_num", "char_str", "int_num",
        "int_num", "int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [], [], [], [], [], [], [], [], [],
        [float("nan")],
        [], [], [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[float("nan")], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), 100001, float("nan"), 100001, 100001, float("nan"),
        100001, float("nan"), float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
)
pipeline_0 = make_pipeline(
    column_selector_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
column_selector_1 = ColumnSelector(
    columns_indices_list=[
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
    ]
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[6])
float_str2_float = FloatStr2Float(
    dtypes_list=["float_num"], missing_values_reference_list=[float("nan")]
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[float("nan")], filling_values=float("nan")
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    column_selector_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 6,
    ],
)
t_no_op = TNoOp(
    fun=None, name="no_action", datatypes="x", feat_constraints=[]
)
random_forest_classifier = RandomForestClassifier(
    class_weight="balanced",
    criterion="entropy",
    max_depth=5,
    max_features=0.9938750513379689,
    min_samples_leaf=5,
    n_estimators=97,
    n_jobs=4,
    random_state=33,
)


#### Pipeline.

In [ ]:
pipeline = make_pipeline(
    union, numpy_permute_array, t_no_op, random_forest_classifier
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X.values, train_y.values.ravel());

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

In [ ]:
pipeline.predict(test_X.values[:10])

# 3. Save model

In [ ]:
model_name = 'ai_governance_test_model'

In [ ]:
fields=train_X.columns.tolist()
target_col = train_y.columns.tolist()[0]
metadata_dict = {'target_col' : target_col,'fields':fields}

In [ ]:
# TODO: 
# adapt the "saving model" from the original "1-model-training-with-factsheets" to this model
# https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/fd15bd20-0d9c-45b2-9b25-50312bab7084/view?projectid=c7354a00-b66b-4896-8443-40b1ac51c846&context=cpdaas


collect a bunch of informations and hand them over to the facts_client

In [ ]:
fields=train_X.columns.tolist()
target_col = train_y.columns.tolist()[0]
metadata_dict = {'target_col' : target_col,'fields':fields}


training_data_references = [
                {
                    "id": "car_insurance_data",
                    "type": "connection_asset",
                    "location": {
                        "select_statement": "training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=False)",
                        "table_name": "car_insurance_data imported from AutoAI"
                    }
                }]

software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")


model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:model_name,
        wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.0",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD:"car_insurance",
        wml_client._models.ConfigurationMetaNames.TAGS: ['car_insurance_outcome_v01'],
        wml_client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.CUSTOM: metadata_dict
    }

facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

#Defining Notebook URL
nb_name = "1-model-training-with-factsheet"
nb_asset_id = "tbd"
nb_asset_url = "https://" + CPD_URL + "/analytics/notebooks/v2/" + nb_asset_id + "?projectid=" + PROJECT_UID + "&context=cpdaas"

latestRunId = facts_client.runs.list_runs_by_experiment('1').sort_values('start_time').iloc[-1]['run_id']
facts_client.runs.set_tags(latestRunId, {"Notebook name": nb_name, "Notebook id": nb_asset_id, "Notebook URL" : nb_asset_url})
facts_client.export_facts.export_payload(latestRunId)

RUN_ID=facts_client.runs.get_current_run_id()
facts_client.export_facts.export_payload(RUN_ID)

In [ ]:
print("Storing model .....")

published_model_details = wml_client.repository.store_model(model=pipeline, 
                                                        meta_props=model_props, 
                                                        training_data=train_X, 
                                                        training_target=train_y)
model_uid = wml_client.repository.get_model_id(published_model_details)
print("The model",model_name,"successfully stored in the project")
print("Model ID: {}".format(model_uid))